In [63]:
import pandas as pd
from sqlalchemy import create_engine, text
import sys

## Configurando pandas
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Função de conexão banco de dados

In [64]:
def connect_to_database(database):
    try:
        connection_string = f'mssql+pyodbc://localhost/{database}?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'
        engine = create_engine(connection_string)
        return engine 
    except Exception as e:
        print("Erro ao conectar ao banco de dados:", e)
        return None

## TbAccount

In [65]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbCabecalho.csv'
dataset = pd.read_csv(file_path, sep='\t')
select_columns = [
    'id_account',
    'account_username',
    'account_biography',
    'profile_picture_url',
    'account_name'
]

dataset= dataset[select_columns]
dataset.head()

,id_account,account_username,account_biography,profile_picture_url,account_name
0,17841425444516188,bodemeier.digital,♟️Criação de conteúdo estratégico\n▪️+ de 5 an...,https://scontent.fcgh8-1.fna.fbcdn.net/v/t51.2...,Bodemeier Digital
1,17841417530400616,gabgalani,RJ 🔸 SP\ndata is the new oil 💻\n@carolinebodem...,https://scontent.fcgh8-1.fna.fbcdn.net/v/t51.2...,Gabriel Galani


In [66]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id_account           2 non-null      int64 
 1   account_username     2 non-null      object
 2   account_biography    2 non-null      object
 3   profile_picture_url  2 non-null      object
 4   account_name         2 non-null      object
dtypes: int64(1), object(4)
memory usage: 212.0+ bytes


In [67]:
d_type = {
    'id_account': object
}
dataset = dataset.astype(d_type)
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id_account           2 non-null      object
 1   account_username     2 non-null      object
 2   account_biography    2 non-null      object
 3   profile_picture_url  2 non-null      object
 4   account_name         2 non-null      object
dtypes: object(5)
memory usage: 212.0+ bytes


Carregar para o banco

In [68]:
dataset.head()

,id_account,account_username,account_biography,profile_picture_url,account_name
0,17841425444516188,bodemeier.digital,♟️Criação de conteúdo estratégico\n▪️+ de 5 an...,https://scontent.fcgh8-1.fna.fbcdn.net/v/t51.2...,Bodemeier Digital
1,17841417530400616,gabgalani,RJ 🔸 SP\ndata is the new oil 💻\n@carolinebodem...,https://scontent.fcgh8-1.fna.fbcdn.net/v/t51.2...,Gabriel Galani


In [69]:
engine = connect_to_database('DW')
table_name = 'TbAccount'
key_column = 'id_account'

if not engine: 
    print('conexão fail')
    sys.exit()

with engine.connect() as connection:
    for i, row in dataset.iterrows():
        query = text(f"select count(*) from {table_name} WHERE {key_column} = :value")
        result = connection.execute(query, parameters=dict(value=row[key_column]))
        count = result.scalar()

        if count > 0:
            print(f'Registro com {key_column} = {row[key_column]} já existe.')
        else: 
            row.to_frame().T.to_sql(table_name, con=connection, if_exists='append', index=False)
            print(f"Registro com {key_column} = {row[key_column]} inserido com sucesso.")
    connection.commit()
    connection.close()

Registro com id_account = 17841425444516188 já existe.
Registro com id_account = 17841417530400616 já existe.


## DTbAccountInsights e FTbAccountsDayInsights e FTbAccountsLifetimeInsights

Coletando do arquivo de dia

In [70]:

file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbAccontDayInsights.csv'
dataset = pd.read_csv(file_path, sep='\t')
select_columns = [
    'name',
    'period',
    'title',
    'description'
]
FTbAccountDayInsights = dataset
dataset_day_dimension = dataset[select_columns]
dataset_day_dimension.head()

,name,period,title,description
0,impressions,day,Impressões,Número total de vezes que os objetos de mídia ...
1,reach,day,Alcance,Número total de vezes que os objetos de mídia ...
2,follower_count,day,Número de seguidores,Número total de contas únicas que seguem este ...
3,email_contacts,day,Contatos de email,Número total de toques no link de email deste ...
4,phone_call_clicks,day,Cliques em ligação telefônica,Número total de toques no link de ligação dest...


Coletando do arquivo de lifetime

In [71]:
cidade_seguidores = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbAccountLifeCidadeDosSeguidores.csv'
Faixa_genero = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbAccountLifeFaixaGenero.csv'
local_pais = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbAccountLifeLocaPais.csv'
pais_seguidores = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbAccountLifePaisDosSeguidores.csv'
seguidores_on = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbAccountLifeSeguidoresOnline.csv'

list_df = [cidade_seguidores, Faixa_genero, local_pais, pais_seguidores, seguidores_on]
dfs = [] 

for df in list_df: 
    data = pd.read_csv(df, sep='\t')
    dfs.append(data)

dataset = pd.concat(dfs)
select_columns = [
    'name',
    'period',
    'title',
    'description'
]
FTbAccountLifetimeInsights = dataset
dataset_lifetime_dimension = dataset[select_columns]
dataset_lifetime_dimension.head()

,name,period,title,description
0,audience_city,lifetime,Cidade do público,Cidades dos seguidores deste perfil
1,audience_city,lifetime,Cidade do público,Cidades dos seguidores deste perfil
2,audience_city,lifetime,Cidade do público,Cidades dos seguidores deste perfil
3,audience_city,lifetime,Cidade do público,Cidades dos seguidores deste perfil
4,audience_city,lifetime,Cidade do público,Cidades dos seguidores deste perfil


ADICIONADO POSTERIORMENTE
Carregando a tabela de insights de mdiias e stories para essa dimensão

In [72]:

file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbStoriesInsights.csv'
dataset_1 = pd.read_csv(file_path, sep='\t')
select_columns = [
    'name',
    'period',
    'title',
    'description'
]
stories_descricao_dimensao = dataset_1[select_columns]
stories_descricao_dimensao.head(10)

,name,period,title,description
0,exits,lifetime,Saídas,Número de vezes que alguém saiu do story
1,replies,lifetime,Respostas,Número total de respostas ao story
2,taps_forward,lifetime,Toques para avançar,Número total de toques para ver a foto ou víde...
3,taps_back,lifetime,Toques para voltar,Número total de toques para ver a foto ou o ví...
4,impressions,lifetime,Impressões,O número de vezes que seu story foi exibido na...
5,reach,lifetime,Contas alcançadas,O número de contas únicas que viram este story...


In [73]:

file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbMidiasInsights.csv'
dataset_2 = pd.read_csv(file_path, sep='\t')
select_columns = [
    'name',
    'period',
    'title',
    'description'
]
midias_descricao_dimensao = dataset_2[select_columns]
midias_descricao_dimensao.head()

,name,period,title,description
0,reach,lifetime,Contas alcançadas,O número de contas únicas que viram este reel ...
1,saved,lifetime,Salvo,O número de vezes que seu reel foi salvo.
2,reach,lifetime,Contas alcançadas,O número de contas únicas que viram este reel ...
3,saved,lifetime,Salvo,O número de vezes que seu reel foi salvo.
4,reach,lifetime,Contas alcançadas,O número de contas únicas que viram este reel ...


Transformando os datasets em dados unicos

In [74]:
dataset_day_dimension.drop_duplicates(inplace=True)
dataset_lifetime_dimension.drop_duplicates(inplace=True)
stories_descricao_dimensao.drop_duplicates(subset=('name'), inplace=True)
midias_descricao_dimensao.drop_duplicates(subset=('name'),inplace=True)

C:\Users\gabri\AppData\Local\Temp\ipykernel_18000\3089048326.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_day_dimension.drop_duplicates(inplace=True)
C:\Users\gabri\AppData\Local\Temp\ipykernel_18000\3089048326.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_lifetime_dimension.drop_duplicates(inplace=True)
C:\Users\gabri\AppData\Local\Temp\ipykernel_18000\3089048326.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

Unindo as dimensões

In [75]:
DTbAccountInsights = pd.concat([dataset_day_dimension,dataset_lifetime_dimension, stories_descricao_dimensao, midias_descricao_dimensao])
DTbAccountInsights.sort_values('name', inplace=True)
DTbAccountInsights.drop_duplicates(subset=('name'),inplace=True)
DTbAccountInsights.reset_index(inplace=True)
DTbAccountInsights['id_insight'] = DTbAccountInsights.index + 1
DTbAccountInsights = DTbAccountInsights[['id_insight'] + [col for col in DTbAccountInsights.columns if col != 'id_insight']]
DTbAccountInsights.drop(columns='index', inplace=True)
DTbAccountInsights.rename(columns={'period': 'frequencia'}, inplace=True)
DTbAccountInsights.head()

,id_insight,name,frequencia,title,description
0,1,audience_city,lifetime,Cidade do público,Cidades dos seguidores deste perfil
1,2,audience_country,lifetime,País do público,Países dos seguidores deste perfil
2,3,audience_gender_age,lifetime,Gênero e faixa etária,A distribuição por gênero e faixa etária dos s...
3,4,audience_locale,lifetime,Localização,Localidades por códigos de país dos seguidores...
4,5,email_contacts,day,Contatos de email,Número total de toques no link de email deste ...


In [76]:
DTbAccountInsights.head(100)

,id_insight,name,frequencia,title,description
0,1,audience_city,lifetime,Cidade do público,Cidades dos seguidores deste perfil
1,2,audience_country,lifetime,País do público,Países dos seguidores deste perfil
2,3,audience_gender_age,lifetime,Gênero e faixa etária,A distribuição por gênero e faixa etária dos s...
3,4,audience_locale,lifetime,Localização,Localidades por códigos de país dos seguidores...
4,5,email_contacts,day,Contatos de email,Número total de toques no link de email deste ...
5,6,engagement,lifetime,Engajamento,"Número total de curtidas, comentários e salvam..."
6,7,exits,lifetime,Saídas,Número de vezes que alguém saiu do story
7,8,follower_count,day,Número de seguidores,Número total de contas únicas que seguem este ...
8,9,get_directions_clicks,day,Cliques em Como chegar,Número total de toques no link Como chegar des...
9,10,impressions,lifetime,Impressões,O número de vezes que seu story foi exibido na...


Carregando no banco DTbAccountInsights

In [77]:
DTbAccountInsights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id_insight   21 non-null     int64 
 1   name         21 non-null     object
 2   frequencia   21 non-null     object
 3   title        21 non-null     object
 4   description  21 non-null     object
dtypes: int64(1), object(4)
memory usage: 972.0+ bytes


In [78]:
engine = connect_to_database('DW')
table_name = 'DTbDescricaoInsights'
key_column = 'id_insight'

if not engine: 
    print('conexão fail')
    sys.exit()

with engine.connect() as connection:
    for i, row in DTbAccountInsights.iterrows():
        query = text(f"select count(*) from {table_name} WHERE {key_column} = :value")
        result = connection.execute(query, parameters=dict(value=row[key_column]))
        count = result.scalar()

        if count > 0:
            print(f'Registro com {key_column} = {row[key_column]} já existe.')
        else: 
            row.to_frame().T.to_sql(table_name, con=connection, if_exists='append', index=False)
            print(f"Registro com {key_column} = {row[key_column]} inserido com sucesso.")
    connection.commit()
    connection.close()

Registro com id_insight = 1 já existe.
Registro com id_insight = 2 já existe.
Registro com id_insight = 3 já existe.
Registro com id_insight = 4 já existe.
Registro com id_insight = 5 já existe.
Registro com id_insight = 6 já existe.
Registro com id_insight = 7 já existe.
Registro com id_insight = 8 já existe.
Registro com id_insight = 9 já existe.
Registro com id_insight = 10 já existe.
Registro com id_insight = 11 já existe.
Registro com id_insight = 12 já existe.
Registro com id_insight = 13 já existe.
Registro com id_insight = 14 já existe.
Registro com id_insight = 15 já existe.
Registro com id_insight = 16 já existe.
Registro com id_insight = 17 já existe.
Registro com id_insight = 18 já existe.
Registro com id_insight = 19 já existe.
Registro com id_insight = 20 já existe.
Registro com id_insight = 21 já existe.


Carregando a tabela FTbAccountDayInsights

In [79]:
FTbAccountDayInsights.head()

,Unnamed: 0,name,period,title,description,id,username,last_day,last_end_time,actual_day,actual_end_time,id_account,chave,extract_date,period_extraction,year,day,id_tb_midia
0,0,impressions,day,Impressões,Número total de vezes que os objetos de mídia ...,17841425444516188/insights/impressions/day,bodemeier.digital,2,2024-05-23T07:00:00+0000,0,2024-05-24T07:00:00+0000,17841425444516188,17841425444516188impressions,2024-05-24,5,2024,24,1784142544451618820240524impressionsday
1,1,reach,day,Alcance,Número total de vezes que os objetos de mídia ...,17841425444516188/insights/reach/day,bodemeier.digital,1,2024-05-23T07:00:00+0000,0,2024-05-24T07:00:00+0000,17841425444516188,17841425444516188reach,2024-05-24,5,2024,24,1784142544451618820240524reachday
2,2,follower_count,day,Número de seguidores,Número total de contas únicas que seguem este ...,17841425444516188/insights/follower_count/day,bodemeier.digital,0,2024-05-23T07:00:00+0000,0,2024-05-24T07:00:00+0000,17841425444516188,17841425444516188follower_count,2024-05-24,5,2024,24,1784142544451618820240524follower_countday
3,3,email_contacts,day,Contatos de email,Número total de toques no link de email deste ...,17841425444516188/insights/email_contacts/day,bodemeier.digital,0,2024-05-23T07:00:00+0000,0,2024-05-24T07:00:00+0000,17841425444516188,17841425444516188email_contacts,2024-05-24,5,2024,24,1784142544451618820240524email_contactsday
4,4,phone_call_clicks,day,Cliques em ligação telefônica,Número total de toques no link de ligação dest...,17841425444516188/insights/phone_call_clicks/day,bodemeier.digital,0,2024-05-23T07:00:00+0000,0,2024-05-24T07:00:00+0000,17841425444516188,17841425444516188phone_call_clicks,2024-05-24,5,2024,24,1784142544451618820240524phone_call_clicksday


In [80]:
engine = connect_to_database('DW')
table_name = 'TbAccount'
key_column = 'id_account'

if not engine: 
    print('conexão fail')
    sys.exit()

with engine.connect() as connection:
    for i, row in dataset.iterrows():
        query = text(f"select count(*) from {table_name} WHERE {key_column} = :value")
        result = connection.execute(query, parameters=dict(value=row[key_column]))
        count = result.scalar()

        if count > 0:
            print(f'Registro com {key_column} = {row[key_column]} já existe.')
        else: 
            row.to_frame().T.to_sql(table_name, con=connection, if_exists='append', index=False)
            print(f"Registro com {key_column} = {row[key_column]} inserido com sucesso.")
    connection.commit()
    connection.close()

Registro com id_account = 17841425444516188 já existe.


Registro com id_account = 17841417530400616 já existe.
Registro com id_account = 17841425444516188 já existe.
Registro com id_account = 17841417530400616 já existe.
Registro com id_account = 17841425444516188 já existe.
Registro com id_account = 17841417530400616 já existe.
Registro com id_account = 17841425444516188 já existe.
Registro com id_account = 17841417530400616 já existe.
Registro com id_account = 17841425444516188 já existe.
Registro com id_account = 17841417530400616 já existe.
Registro com id_account = 17841425444516188 já existe.
Registro com id_account = 17841417530400616 já existe.
Registro com id_account = 17841425444516188 já existe.
Registro com id_account = 17841417530400616 já existe.
Registro com id_account = 17841425444516188 já existe.
Registro com id_account = 17841417530400616 já existe.
Registro com id_account = 17841425444516188 já existe.
Registro com id_account = 17841417530400616 já existe.
Registro com id_account = 17841425444516188 já existe.
Registro c

Tabela FTbAccountDayInsights

Unindo com a dimensao de DTbAccountInsights para trazer o id

In [81]:
FTbAccountDayInsights = pd.merge(FTbAccountDayInsights, DTbAccountInsights, on='name', how= 'inner')
FTbAccountDayInsights.head(1)

,Unnamed: 0,name,period,title_x,description_x,id,username,last_day,last_end_time,actual_day,actual_end_time,id_account,chave,extract_date,period_extraction,year,day,id_tb_midia,id_insight,frequencia,title_y,description_y
0,0,impressions,day,Impressões,Número total de vezes que os objetos de mídia ...,17841425444516188/insights/impressions/day,bodemeier.digital,2,2024-05-23T07:00:00+0000,0,2024-05-24T07:00:00+0000,17841425444516188,17841425444516188impressions,2024-05-24,5,2024,24,1784142544451618820240524impressionsday,10,lifetime,Impressões,O número de vezes que seu story foi exibido na...


In [82]:
select_columns = [
  'id_tb_midia',
  'id_insight',
  'id_account',
  'last_day',
  'last_end_time',
  'actual_day',
  'actual_end_time',
  'extract_date',
  'period_extraction',
  'year',
  'day'
 ]
FTbAccountDayInsights = FTbAccountDayInsights[select_columns]
FTbAccountDayInsights.head()

,id_tb_midia,id_insight,id_account,last_day,last_end_time,actual_day,actual_end_time,extract_date,period_extraction,year,day
0,1784142544451618820240524impressionsday,10,17841425444516188,2,2024-05-23T07:00:00+0000,0,2024-05-24T07:00:00+0000,2024-05-24,5,2024,24
1,1784142544451618820240524reachday,14,17841425444516188,1,2024-05-23T07:00:00+0000,0,2024-05-24T07:00:00+0000,2024-05-24,5,2024,24
2,1784142544451618820240524follower_countday,8,17841425444516188,0,2024-05-23T07:00:00+0000,0,2024-05-24T07:00:00+0000,2024-05-24,5,2024,24
3,1784142544451618820240524email_contactsday,5,17841425444516188,0,2024-05-23T07:00:00+0000,0,2024-05-24T07:00:00+0000,2024-05-24,5,2024,24
4,1784142544451618820240524phone_call_clicksday,12,17841425444516188,0,2024-05-23T07:00:00+0000,0,2024-05-24T07:00:00+0000,2024-05-24,5,2024,24


Renomeando colunas

In [83]:
columns_rename = {
    'id_tb_midia': 'id',
    'last_day': 'value_last_day',
    'actual_day': 'value_actual_day'
}
FTbAccountDayInsights.rename(columns = columns_rename, inplace=True)
FTbAccountDayInsights.head()

,id,id_insight,id_account,value_last_day,last_end_time,value_actual_day,actual_end_time,extract_date,period_extraction,year,day
0,1784142544451618820240524impressionsday,10,17841425444516188,2,2024-05-23T07:00:00+0000,0,2024-05-24T07:00:00+0000,2024-05-24,5,2024,24
1,1784142544451618820240524reachday,14,17841425444516188,1,2024-05-23T07:00:00+0000,0,2024-05-24T07:00:00+0000,2024-05-24,5,2024,24
2,1784142544451618820240524follower_countday,8,17841425444516188,0,2024-05-23T07:00:00+0000,0,2024-05-24T07:00:00+0000,2024-05-24,5,2024,24
3,1784142544451618820240524email_contactsday,5,17841425444516188,0,2024-05-23T07:00:00+0000,0,2024-05-24T07:00:00+0000,2024-05-24,5,2024,24
4,1784142544451618820240524phone_call_clicksday,12,17841425444516188,0,2024-05-23T07:00:00+0000,0,2024-05-24T07:00:00+0000,2024-05-24,5,2024,24


Tipagem de dados

In [84]:
FTbAccountDayInsights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 18 non-null     object
 1   id_insight         18 non-null     int64 
 2   id_account         18 non-null     int64 
 3   value_last_day     18 non-null     int64 
 4   last_end_time      18 non-null     object
 5   value_actual_day   18 non-null     int64 
 6   actual_end_time    18 non-null     object
 7   extract_date       18 non-null     object
 8   period_extraction  18 non-null     int64 
 9   year               18 non-null     int64 
 10  day                18 non-null     int64 
dtypes: int64(7), object(4)
memory usage: 1.7+ KB


In [85]:
# Primeiro, vamos converter as colunas de data para o tipo de dados datetime64
FTbAccountDayInsights['last_end_time'] = pd.to_datetime(FTbAccountDayInsights['last_end_time'])
FTbAccountDayInsights['actual_end_time'] = pd.to_datetime(FTbAccountDayInsights['actual_end_time'])
FTbAccountDayInsights['extract_date'] = pd.to_datetime(FTbAccountDayInsights['extract_date'])

# Agora, vamos extrair apenas a data (sem a hora) das colunas de data
FTbAccountDayInsights['last_end_time'] = FTbAccountDayInsights['last_end_time'].dt.date
FTbAccountDayInsights['actual_end_time'] = FTbAccountDayInsights['actual_end_time'].dt.date
FTbAccountDayInsights['extract_date'] = FTbAccountDayInsights['extract_date'].dt.date

type_columns = {
    'period_extraction': 'object',
    'year': 'object',
    'day': 'object'                
}

FTbAccountDayInsights = FTbAccountDayInsights.astype(type_columns)

In [86]:
FTbAccountDayInsights.head()

,id,id_insight,id_account,value_last_day,last_end_time,value_actual_day,actual_end_time,extract_date,period_extraction,year,day
0,1784142544451618820240524impressionsday,10,17841425444516188,2,2024-05-23,0,2024-05-24,2024-05-24,5,2024,24
1,1784142544451618820240524reachday,14,17841425444516188,1,2024-05-23,0,2024-05-24,2024-05-24,5,2024,24
2,1784142544451618820240524follower_countday,8,17841425444516188,0,2024-05-23,0,2024-05-24,2024-05-24,5,2024,24
3,1784142544451618820240524email_contactsday,5,17841425444516188,0,2024-05-23,0,2024-05-24,2024-05-24,5,2024,24
4,1784142544451618820240524phone_call_clicksday,12,17841425444516188,0,2024-05-23,0,2024-05-24,2024-05-24,5,2024,24


Carregando os dados no banco

In [87]:
engine = connect_to_database('DW')
table_name = 'FTbAccountDayInsights'
key_column = 'id'

if not engine: 
    print('conexão fail')
    sys.exit()

with engine.connect() as connection:
    for i, row in FTbAccountDayInsights.iterrows():
        query = text(f"select count(*) from {table_name} WHERE {key_column} = :value")
        result = connection.execute(query, parameters=dict(value=row[key_column]))
        count = result.scalar()

        if count > 0:
            print(f'Registro com {key_column} = {row[key_column]} já existe.')
        else: 
            row.to_frame().T.to_sql(table_name, con=connection, if_exists='append', index=False)
            print(f"Registro com {key_column} = {row[key_column]} inserido com sucesso.")
    connection.commit()
    connection.close()

Registro com id = 1784142544451618820240524impressionsday já existe.
Registro com id = 1784142544451618820240524reachday já existe.
Registro com id = 1784142544451618820240524follower_countday já existe.
Registro com id = 1784142544451618820240524email_contactsday já existe.
Registro com id = 1784142544451618820240524phone_call_clicksday já existe.
Registro com id = 1784142544451618820240524text_message_clicksday já existe.
Registro com id = 1784142544451618820240524get_directions_clicksday já existe.
Registro com id = 1784142544451618820240524website_clicksday já existe.
Registro com id = 1784142544451618820240524profile_viewsday já existe.
Registro com id = 1784141753040061620240524impressionsday já existe.
Registro com id = 1784141753040061620240524reachday já existe.
Registro com id = 1784141753040061620240524follower_countday já existe.
Registro com id = 1784141753040061620240524email_contactsday já existe.
Registro com id = 1784141753040061620240524phone_call_clicksday já existe.

Tabela FTbAccountLifetimeInsights

Unindo com a dimensao de DTbAccountInsights para trazer o id

In [88]:
FTbAccountLifetimeInsights.head()

,Unnamed: 0,name,period,title,description,username,id_account,extract_date,period_extraction,year,day,id_tb_account,age_gender,value
0,0,audience_city,lifetime,Cidade do público,Cidades dos seguidores deste perfil,bodemeier.digital,17841425444516188,2024-05-24,5,2024,24,1784142544451618820240524audience_citylifetime,"value.São Paulo, São Paulo (state)",180.0
1,1,audience_city,lifetime,Cidade do público,Cidades dos seguidores deste perfil,gabgalani,17841417530400616,2024-05-24,5,2024,24,1784141753040061620240524audience_citylifetime,"value.São Paulo, São Paulo (state)",108.0
2,2,audience_city,lifetime,Cidade do público,Cidades dos seguidores deste perfil,bodemeier.digital,17841425444516188,2024-05-24,5,2024,24,1784142544451618820240524audience_citylifetime,"value.Karabük, Karabük Province",8.0
3,3,audience_city,lifetime,Cidade do público,Cidades dos seguidores deste perfil,gabgalani,17841417530400616,2024-05-24,5,2024,24,1784141753040061620240524audience_citylifetime,"value.Karabük, Karabük Province",NaN
4,4,audience_city,lifetime,Cidade do público,Cidades dos seguidores deste perfil,bodemeier.digital,17841425444516188,2024-05-24,5,2024,24,1784142544451618820240524audience_citylifetime,"value.Rio de Janeiro, Rio de Janeiro (state)",24.0


In [89]:
FTbAccountLifetimeInsights = pd.merge(FTbAccountLifetimeInsights, DTbAccountInsights, on='name', how= 'inner')
FTbAccountLifetimeInsights.head(1)

,Unnamed: 0,name,period,title_x,description_x,username,id_account,extract_date,period_extraction,year,day,id_tb_account,age_gender,value,id_insight,frequencia,title_y,description_y
0,0,audience_city,lifetime,Cidade do público,Cidades dos seguidores deste perfil,bodemeier.digital,17841425444516188,2024-05-24,5,2024,24,1784142544451618820240524audience_citylifetime,"value.São Paulo, São Paulo (state)",180.0,1,lifetime,Cidade do público,Cidades dos seguidores deste perfil


Selecionando colunas

In [90]:
select_columns = [
    'id_tb_account',
    'id_account',
    'id_insight',
    'extract_date',
    'period_extraction',
    'year',
    'day',
    'name',
    'age_gender',
    'value'
]

FTbAccountLifetimeInsights = FTbAccountLifetimeInsights[select_columns]
FTbAccountLifetimeInsights.head()

,id_tb_account,id_account,id_insight,extract_date,period_extraction,year,day,name,age_gender,value
0,1784142544451618820240524audience_citylifetime,17841425444516188,1,2024-05-24,5,2024,24,audience_city,"value.São Paulo, São Paulo (state)",180.0
1,1784141753040061620240524audience_citylifetime,17841417530400616,1,2024-05-24,5,2024,24,audience_city,"value.São Paulo, São Paulo (state)",108.0
2,1784142544451618820240524audience_citylifetime,17841425444516188,1,2024-05-24,5,2024,24,audience_city,"value.Karabük, Karabük Province",8.0
3,1784141753040061620240524audience_citylifetime,17841417530400616,1,2024-05-24,5,2024,24,audience_city,"value.Karabük, Karabük Province",NaN
4,1784142544451618820240524audience_citylifetime,17841425444516188,1,2024-05-24,5,2024,24,audience_city,"value.Rio de Janeiro, Rio de Janeiro (state)",24.0


Renomando colunas

In [91]:
rename_columns = {
    'id_tb_account': 'id',
    'name': 'definicao',
    'age_gender': 'value_descricao'
}

FTbAccountLifetimeInsights.rename(columns=rename_columns, inplace=True)
FTbAccountLifetimeInsights['value_descricao'] = FTbAccountLifetimeInsights['value_descricao'].str.replace('value.', '')
FTbAccountLifetimeInsights.head(5)


,id,id_account,id_insight,extract_date,period_extraction,year,day,definicao,value_descricao,value
0,1784142544451618820240524audience_citylifetime,17841425444516188,1,2024-05-24,5,2024,24,audience_city,"São Paulo, São Paulo (state)",180.0
1,1784141753040061620240524audience_citylifetime,17841417530400616,1,2024-05-24,5,2024,24,audience_city,"São Paulo, São Paulo (state)",108.0
2,1784142544451618820240524audience_citylifetime,17841425444516188,1,2024-05-24,5,2024,24,audience_city,"Karabük, Karabük Province",8.0
3,1784141753040061620240524audience_citylifetime,17841417530400616,1,2024-05-24,5,2024,24,audience_city,"Karabük, Karabük Province",NaN
4,1784142544451618820240524audience_citylifetime,17841425444516188,1,2024-05-24,5,2024,24,audience_city,"Rio de Janeiro, Rio de Janeiro (state)",24.0


Tratando valores que não são numeros

In [125]:
def replace_non_numeric(value):
    try:
        float(value)  # Tenta converter o valor para float
        return value  # Se for numérico, mantenha o valor
    except ValueError:
        return 0  # Se não for numérico, substitua por 0

# Aplique a função à coluna 'value'
FTbAccountLifetimeInsights['value'] = FTbAccountLifetimeInsights['value'].apply(lambda x: replace_non_numeric(x))
FTbAccountLifetimeInsights['value'] = FTbAccountLifetimeInsights['value'].fillna(0)
FTbAccountLifetimeInsights.head()

,id,id_account,id_insight,extract_date,period_extraction,year,day,definicao,value_descricao,value
0,"1784142544451618812024-05-24São Paulo, São Pau...",17841425444516188,1,2024-05-24,5,2024,24,audience_city,"São Paulo, São Paulo (state)",180.0
1,"1784141753040061612024-05-24São Paulo, São Pau...",17841417530400616,1,2024-05-24,5,2024,24,audience_city,"São Paulo, São Paulo (state)",108.0
2,"1784142544451618812024-05-24Karabük, Karabük P...",17841425444516188,1,2024-05-24,5,2024,24,audience_city,"Karabük, Karabük Province",8.0
4,"1784142544451618812024-05-24Rio de Janeiro, Ri...",17841425444516188,1,2024-05-24,5,2024,24,audience_city,"Rio de Janeiro, Rio de Janeiro (state)",24.0
5,"1784141753040061612024-05-24Rio de Janeiro, Ri...",17841417530400616,1,2024-05-24,5,2024,24,audience_city,"Rio de Janeiro, Rio de Janeiro (state)",7.0


Tipagem dos dados

In [93]:
FTbAccountLifetimeInsights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338 entries, 0 to 337
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 338 non-null    object 
 1   id_account         338 non-null    int64  
 2   id_insight         338 non-null    int64  
 3   extract_date       338 non-null    object 
 4   period_extraction  338 non-null    int64  
 5   year               338 non-null    int64  
 6   day                338 non-null    int64  
 7   definicao          338 non-null    object 
 8   value_descricao    338 non-null    object 
 9   value              338 non-null    float64
dtypes: float64(1), int64(5), object(4)
memory usage: 26.5+ KB


In [94]:
FTbAccountLifetimeInsights['extract_date'] = pd.to_datetime(FTbAccountLifetimeInsights['extract_date'])

type_columns = {
    'id_account': 'object',
    'period_extraction': 'object',
    'year': 'object',
    'day': 'object'
}

FTbAccountLifetimeInsights = FTbAccountLifetimeInsights.astype(type_columns)
FTbAccountLifetimeInsights.drop(columns='id', inplace=True)
FTbAccountLifetimeInsights['id'] = FTbAccountLifetimeInsights['id_account'].astype(str) + FTbAccountLifetimeInsights['id_insight'].astype(str) +  FTbAccountLifetimeInsights['extract_date'].astype(str) + FTbAccountLifetimeInsights['value_descricao'] 
FTbAccountLifetimeInsights.head()

,id_account,id_insight,extract_date,period_extraction,year,day,definicao,value_descricao,value,id
0,17841425444516188,1,2024-05-24,5,2024,24,audience_city,"São Paulo, São Paulo (state)",180.0,"1784142544451618812024-05-24São Paulo, São Pau..."
1,17841417530400616,1,2024-05-24,5,2024,24,audience_city,"São Paulo, São Paulo (state)",108.0,"1784141753040061612024-05-24São Paulo, São Pau..."
2,17841425444516188,1,2024-05-24,5,2024,24,audience_city,"Karabük, Karabük Province",8.0,"1784142544451618812024-05-24Karabük, Karabük P..."
3,17841417530400616,1,2024-05-24,5,2024,24,audience_city,"Karabük, Karabük Province",0.0,"1784141753040061612024-05-24Karabük, Karabük P..."
4,17841425444516188,1,2024-05-24,5,2024,24,audience_city,"Rio de Janeiro, Rio de Janeiro (state)",24.0,"1784142544451618812024-05-24Rio de Janeiro, Ri..."


In [95]:
colunas = ['id'] + [coluna for coluna in FTbAccountLifetimeInsights if coluna != 'id']
FTbAccountLifetimeInsights = FTbAccountLifetimeInsights[colunas]
FTbAccountLifetimeInsights.head()

,id,id_account,id_insight,extract_date,period_extraction,year,day,definicao,value_descricao,value
0,"1784142544451618812024-05-24São Paulo, São Pau...",17841425444516188,1,2024-05-24,5,2024,24,audience_city,"São Paulo, São Paulo (state)",180.0
1,"1784141753040061612024-05-24São Paulo, São Pau...",17841417530400616,1,2024-05-24,5,2024,24,audience_city,"São Paulo, São Paulo (state)",108.0
2,"1784142544451618812024-05-24Karabük, Karabük P...",17841425444516188,1,2024-05-24,5,2024,24,audience_city,"Karabük, Karabük Province",8.0
3,"1784141753040061612024-05-24Karabük, Karabük P...",17841417530400616,1,2024-05-24,5,2024,24,audience_city,"Karabük, Karabük Province",0.0
4,"1784142544451618812024-05-24Rio de Janeiro, Ri...",17841425444516188,1,2024-05-24,5,2024,24,audience_city,"Rio de Janeiro, Rio de Janeiro (state)",24.0


Carregando apenas linnhas que tem valores

In [126]:
FTbAccountLifetimeInsights = FTbAccountLifetimeInsights[FTbAccountLifetimeInsights['value'] != 0]
FTbAccountLifetimeInsights.info()

<class 'pandas.core.frame.DataFrame'>
Index: 195 entries, 0 to 335
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   id                 195 non-null    object        
 1   id_account         195 non-null    object        
 2   id_insight         195 non-null    int64         
 3   extract_date       195 non-null    datetime64[ns]
 4   period_extraction  195 non-null    object        
 5   year               195 non-null    object        
 6   day                195 non-null    object        
 7   definicao          195 non-null    object        
 8   value_descricao    195 non-null    object        
 9   value              195 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(1), object(7)
memory usage: 16.8+ KB


In [97]:
FTbAccountLifetimeInsights.drop_duplicates(inplace=True)

Carregando no banco

In [98]:
engine = connect_to_database('DW')
table_name = 'FTbAccountLifetimeInsights'
key_column = 'id'

if not engine: 
    print('conexão fail')
    sys.exit()

with engine.connect() as connection:
    for i, row in FTbAccountLifetimeInsights.iterrows():
        query = text(f"select count(*) from {table_name} WHERE {key_column} = :value")
        result = connection.execute(query, parameters=dict(value=row[key_column]))
        count = result.scalar()

        try:
            if count > 0:
                print(f'Registro com {key_column} = {row[key_column]} já existe.')
            else: 
                row.to_frame().T.to_sql(table_name, con=connection, if_exists='append', index=False)
                print(f"Registro com {key_column} = {row[key_column]} inserido com sucesso.")
        except:
            pass
    connection.commit()
    connection.close()

Registro com id = 1784142544451618812024-05-24São Paulo, São Paulo (state) já existe.


Registro com id = 1784141753040061612024-05-24São Paulo, São Paulo (state) já existe.
Registro com id = 1784142544451618812024-05-24Karabük, Karabük Province já existe.
Registro com id = 1784142544451618812024-05-24Rio de Janeiro, Rio de Janeiro (state) já existe.
Registro com id = 1784141753040061612024-05-24Rio de Janeiro, Rio de Janeiro (state) já existe.
Registro com id = 1784142544451618812024-05-24Yuma, Arizona já existe.
Registro com id = 1784142544451618812024-05-24Edinburg, Texas já existe.
Registro com id = 1784142544451618812024-05-24Abington, Pennsylvania já existe.
Registro com id = 1784142544451618812024-05-24Sioux Falls, South Dakota já existe.
Registro com id = 1784142544451618812024-05-24Maceió, Alagoas já existe.
Registro com id = 1784142544451618812024-05-24Prague, Prague já existe.
Registro com id = 1784142544451618812024-05-24San Jose, California já existe.
Registro com id = 1784142544451618812024-05-24Redford, Michigan já existe.
Registro com id = 1784142544451618

## Tabelas de midias

In [99]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbMedias.csv'
dataset = pd.read_csv(file_path, sep='\t')
FTbMidias = dataset
dataset.head()

,Unnamed: 0,id_account,username,username.1,id_midia,comments_count,like_count,media_type,media_url,caption,timestamp,permalink,media_product_type,thumbnail_url,shortcode,extract_date,period,year,day,id_tb_midia
0,0,17841425444516188,bodemeier.digital,bodemeier.digital,18005273918369940,2,15,VIDEO,https://scontent.cdninstagram.com/o1/v/t16/f1/...,Você sabe o que é um briefing? 📝\n\n🎥 Assista ...,2024-02-20 21:19:22+00:00,https://www.instagram.com/reel/C3lW1XkOcfL/,REELS,https://scontent.cdninstagram.com/v/t51.29350-...,C3lW1XkOcfL,2024-05-24,5,2024,24,1800527391836994020240524
1,1,17841425444516188,bodemeier.digital,bodemeier.digital,17846370906157882,1,13,VIDEO,https://scontent.cdninstagram.com/o1/v/t16/f1/...,Uma grande dúvida que vocês me perguntam: SOCI...,2024-02-16 21:00:00+00:00,https://www.instagram.com/reel/C3bDRy3uifD/,REELS,https://scontent.cdninstagram.com/v/t51.29350-...,C3bDRy3uifD,2024-05-24,5,2024,24,1784637090615788220240524
2,2,17841425444516188,bodemeier.digital,bodemeier.digital,18002752616172667,0,5,VIDEO,https://scontent.cdninstagram.com/o1/v/t16/f1/...,“O maior instrumento da globalização cultural ...,2024-02-10 20:05:55+00:00,https://www.instagram.com/reel/C3Lf9wLOhpf/,REELS,https://scontent.cdninstagram.com/v/t51.29350-...,C3Lf9wLOhpf,2024-05-24,5,2024,24,1800275261617266720240524
3,3,17841425444516188,bodemeier.digital,bodemeier.digital,17971594208670851,0,7,VIDEO,NaN,‼️ É isso que o Instagram prioriza!\n\nAcredit...,2024-02-09 18:00:00+00:00,https://www.instagram.com/reel/C3KvokquZdf/,REELS,https://scontent.cdninstagram.com/v/t51.29350-...,C3KvokquZdf,2024-05-24,5,2024,24,1797159420867085120240524
4,4,17841425444516188,bodemeier.digital,bodemeier.digital,18062694892504235,5,7,VIDEO,https://scontent.cdninstagram.com/o1/v/t16/f1/...,✨Aqui está: \n\nMas primeiro já me segue para ...,2024-02-09 13:00:00+00:00,https://www.instagram.com/reel/C3IK23sudbS/,REELS,https://scontent.cdninstagram.com/v/t51.29350-...,C3IK23sudbS,2024-05-24,5,2024,24,1806269489250423520240524


In [100]:
select_columns = [
    'id_midia',
    'id_account',
    'media_type',
    'media_url',
    'caption',
    'timestamp',
    'permalink',
    'media_product_type',
    'thumbnail_url',
    'shortcode'
]
DTbMidias = dataset[select_columns]
DTbMidias.head()

,id_midia,id_account,media_type,media_url,caption,timestamp,permalink,media_product_type,thumbnail_url,shortcode
0,18005273918369940,17841425444516188,VIDEO,https://scontent.cdninstagram.com/o1/v/t16/f1/...,Você sabe o que é um briefing? 📝\n\n🎥 Assista ...,2024-02-20 21:19:22+00:00,https://www.instagram.com/reel/C3lW1XkOcfL/,REELS,https://scontent.cdninstagram.com/v/t51.29350-...,C3lW1XkOcfL
1,17846370906157882,17841425444516188,VIDEO,https://scontent.cdninstagram.com/o1/v/t16/f1/...,Uma grande dúvida que vocês me perguntam: SOCI...,2024-02-16 21:00:00+00:00,https://www.instagram.com/reel/C3bDRy3uifD/,REELS,https://scontent.cdninstagram.com/v/t51.29350-...,C3bDRy3uifD
2,18002752616172667,17841425444516188,VIDEO,https://scontent.cdninstagram.com/o1/v/t16/f1/...,“O maior instrumento da globalização cultural ...,2024-02-10 20:05:55+00:00,https://www.instagram.com/reel/C3Lf9wLOhpf/,REELS,https://scontent.cdninstagram.com/v/t51.29350-...,C3Lf9wLOhpf
3,17971594208670851,17841425444516188,VIDEO,NaN,‼️ É isso que o Instagram prioriza!\n\nAcredit...,2024-02-09 18:00:00+00:00,https://www.instagram.com/reel/C3KvokquZdf/,REELS,https://scontent.cdninstagram.com/v/t51.29350-...,C3KvokquZdf
4,18062694892504235,17841425444516188,VIDEO,https://scontent.cdninstagram.com/o1/v/t16/f1/...,✨Aqui está: \n\nMas primeiro já me segue para ...,2024-02-09 13:00:00+00:00,https://www.instagram.com/reel/C3IK23sudbS/,REELS,https://scontent.cdninstagram.com/v/t51.29350-...,C3IK23sudbS


Trabalhando a tipagem dos dados

In [101]:
DTbMidias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id_midia            241 non-null    int64 
 1   id_account          241 non-null    int64 
 2   media_type          241 non-null    object
 3   media_url           233 non-null    object
 4   caption             240 non-null    object
 5   timestamp           241 non-null    object
 6   permalink           241 non-null    object
 7   media_product_type  241 non-null    object
 8   thumbnail_url       36 non-null     object
 9   shortcode           241 non-null    object
dtypes: int64(2), object(8)
memory usage: 19.0+ KB


In [102]:
# Primeiro, vamos converter as colunas de data para o tipo de dados datetime64
DTbMidias['timestamp'] = pd.to_datetime(DTbMidias['timestamp'])

# Agora, vamos extrair apenas a data (sem a hora) das colunas de data
DTbMidias['timestamp'] = DTbMidias['timestamp'].dt.date

type_columns = {
    'id_midia':  'object',
    'id_account': 'object',
}
DTbMidias = DTbMidias.astype(type_columns)
DTbMidias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id_midia            241 non-null    object
 1   id_account          241 non-null    object
 2   media_type          241 non-null    object
 3   media_url           233 non-null    object
 4   caption             240 non-null    object
 5   timestamp           241 non-null    object
 6   permalink           241 non-null    object
 7   media_product_type  241 non-null    object
 8   thumbnail_url       36 non-null     object
 9   shortcode           241 non-null    object
dtypes: object(10)
memory usage: 19.0+ KB


C:\Users\gabri\AppData\Local\Temp\ipykernel_18000\3074123614.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DTbMidias['timestamp'] = pd.to_datetime(DTbMidias['timestamp'])
C:\Users\gabri\AppData\Local\Temp\ipykernel_18000\3074123614.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DTbMidias['timestamp'] = DTbMidias['timestamp'].dt.date


Carregando para o banco

In [103]:
engine = connect_to_database('DW')
table_name = 'DTbMidias'
key_column = 'id_midia'

if not engine: 
    print('conexão fail')
    sys.exit()

with engine.connect() as connection:
    for i, row in DTbMidias.iterrows():
        query = text(f"select count(*) from {table_name} WHERE {key_column} = :value")
        result = connection.execute(query, parameters=dict(value=row[key_column]))
        count = result.scalar()

        if count > 0:
            print(f'Registro com {key_column} = {row[key_column]} já existe.')
        else: 
            row.to_frame().T.to_sql(table_name, con=connection, if_exists='append', index=False)
            print(f"Registro com {key_column} = {row[key_column]} inserido com sucesso.")
    connection.commit()
    connection.close()

Registro com id_midia = 18005273918369940 já existe.
Registro com id_midia = 17846370906157882 já existe.
Registro com id_midia = 18002752616172667 já existe.
Registro com id_midia = 17971594208670851 já existe.
Registro com id_midia = 18062694892504235 já existe.
Registro com id_midia = 17870243565059503 já existe.
Registro com id_midia = 17897388878957061 já existe.
Registro com id_midia = 18214911715272967 já existe.
Registro com id_midia = 17905338809918494 já existe.
Registro com id_midia = 17912491130893668 já existe.
Registro com id_midia = 18020750125974446 já existe.
Registro com id_midia = 17869447275058315 já existe.
Registro com id_midia = 18030343753699664 já existe.
Registro com id_midia = 18012618164496765 já existe.
Registro com id_midia = 18020150257990451 já existe.
Registro com id_midia = 18024592819898084 já existe.
Registro com id_midia = 17997324461240753 já existe.
Registro com id_midia = 17982183917471535 já existe.
Registro com id_midia = 17873173155025720 já e

## Tabela de Stories (Também carrega a tabela de DTbMidias)

In [104]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbStories.csv'
dataset = pd.read_csv(file_path, sep='\t')
stories = dataset
dataset.head()

,Unnamed: 0,id_midia,comments_count,like_count,media_type,media_url,caption,timestamp,permalink,media_product_type,shortcode,username,id_account,username.1,extract_date,period,year,day,id_tb_midia
0,0,18029303027098233,0.0,0.0,IMAGE,https://scontent.cdninstagram.com/v/t51.29350-...,@carolinebodemeier,2024-05-23 10:14:38+00:00,https://instagram.com/stories/gabgalani/337422...,STORY,C7Tqf9UuJmY,gabgalani,17841417530400616,gabgalani,2024-05-23,5,2024,23,1802930302709823320240523


Selecionando as colunas

In [105]:
select_columns = [
    'id_midia',
    'id_account',
    'media_type',
    'media_url',
    'caption',
    'timestamp',
    'permalink',
    'media_product_type',
    'thumbnail_url',
    'shortcode'
]

if 'thumbnail_url' not in dataset.columns:
    dataset['thumbnail_url'] = pd.Series(dtype=object)

DTbMidias = dataset[select_columns]
dataset.head()

,Unnamed: 0,id_midia,comments_count,like_count,media_type,media_url,caption,timestamp,permalink,media_product_type,shortcode,username,id_account,username.1,extract_date,period,year,day,id_tb_midia,thumbnail_url
0,0,18029303027098233,0.0,0.0,IMAGE,https://scontent.cdninstagram.com/v/t51.29350-...,@carolinebodemeier,2024-05-23 10:14:38+00:00,https://instagram.com/stories/gabgalani/337422...,STORY,C7Tqf9UuJmY,gabgalani,17841417530400616,gabgalani,2024-05-23,5,2024,23,1802930302709823320240523,NaN


Trabalhando a tipagem dos dados

In [106]:
DTbMidias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id_midia            1 non-null      int64 
 1   id_account          1 non-null      int64 
 2   media_type          1 non-null      object
 3   media_url           1 non-null      object
 4   caption             1 non-null      object
 5   timestamp           1 non-null      object
 6   permalink           1 non-null      object
 7   media_product_type  1 non-null      object
 8   thumbnail_url       0 non-null      object
 9   shortcode           1 non-null      object
dtypes: int64(2), object(8)
memory usage: 212.0+ bytes


In [107]:
# Primeiro, vamos converter as colunas de data para o tipo de dados datetime64
DTbMidias['timestamp'] = pd.to_datetime(DTbMidias['timestamp'])

# Agora, vamos extrair apenas a data (sem a hora) das colunas de data
DTbMidias['timestamp'] = DTbMidias['timestamp'].dt.date

type_columns = {
    'id_midia':  'object',
    'id_account': 'object',
}
DTbMidias = DTbMidias.astype(type_columns)
DTbMidias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id_midia            1 non-null      object
 1   id_account          1 non-null      object
 2   media_type          1 non-null      object
 3   media_url           1 non-null      object
 4   caption             1 non-null      object
 5   timestamp           1 non-null      object
 6   permalink           1 non-null      object
 7   media_product_type  1 non-null      object
 8   thumbnail_url       0 non-null      object
 9   shortcode           1 non-null      object
dtypes: object(10)
memory usage: 212.0+ bytes


C:\Users\gabri\AppData\Local\Temp\ipykernel_18000\3074123614.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DTbMidias['timestamp'] = pd.to_datetime(DTbMidias['timestamp'])
C:\Users\gabri\AppData\Local\Temp\ipykernel_18000\3074123614.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DTbMidias['timestamp'] = DTbMidias['timestamp'].dt.date


In [108]:
engine = connect_to_database('DW')
table_name = 'DTbMidias'
key_column = 'id_midia'

if not engine: 
    print('conexão fail')
    sys.exit()

with engine.connect() as connection:
    for i, row in DTbMidias.iterrows():
        query = text(f"select count(*) from {table_name} WHERE {key_column} = :value")
        result = connection.execute(query, parameters=dict(value=row[key_column]))
        count = result.scalar()

        if count > 0:
            print(f'Registro com {key_column} = {row[key_column]} já existe.')
        else: 
            row.to_frame().T.to_sql(table_name, con=connection, if_exists='append', index=False)
            print(f"Registro com {key_column} = {row[key_column]} inserido com sucesso.")
    connection.commit()
    connection.close()

Registro com id_midia = 18029303027098233 já existe.


## Tb FTbMidias 

Vou unir o dataframe da TbMidas e dos stories para subir na mesa tabela pois ambos se tratam de midas

Vou normalizar as colunas de ambos os dfs

In [109]:
FTbMidias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          241 non-null    int64 
 1   id_account          241 non-null    int64 
 2   username            241 non-null    object
 3   username.1          241 non-null    object
 4   id_midia            241 non-null    int64 
 5   comments_count      241 non-null    int64 
 6   like_count          241 non-null    int64 
 7   media_type          241 non-null    object
 8   media_url           233 non-null    object
 9   caption             240 non-null    object
 10  timestamp           241 non-null    object
 11  permalink           241 non-null    object
 12  media_product_type  241 non-null    object
 13  thumbnail_url       36 non-null     object
 14  shortcode           241 non-null    object
 15  extract_date        241 non-null    object
 16  period              241 no

In [110]:
FTbMidias.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 241 entries, 0 to 240
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          241 non-null    int64 
 1   id_account          241 non-null    int64 
 2   username            241 non-null    object
 3   username.1          241 non-null    object
 4   id_midia            241 non-null    int64 
 5   comments_count      241 non-null    int64 
 6   like_count          241 non-null    int64 
 7   media_type          241 non-null    object
 8   media_url           233 non-null    object
 9   caption             240 non-null    object
 10  timestamp           241 non-null    object
 11  permalink           241 non-null    object
 12  media_product_type  241 non-null    object
 13  thumbnail_url       36 non-null     object
 14  shortcode           241 non-null    object
 15  extract_date        241 non-null    object
 16  period              241 no

In [111]:
select_columns = [
    'id_midia',
    'id_account',
    'comments_count',
    'like_count',
    'extract_date',
    'period',
    'year',
    'day'
]

df_1 = FTbMidias[select_columns]
df_2 = stories[select_columns]
FTbMidias__ = pd.concat([df_1, df_2], ignore_index=True)
FTbMidias__.head()

,id_midia,id_account,comments_count,like_count,extract_date,period,year,day
0,18005273918369940,17841425444516188,2.0,15.0,2024-05-24,5,2024,24
1,17846370906157882,17841425444516188,1.0,13.0,2024-05-24,5,2024,24
2,18002752616172667,17841425444516188,0.0,5.0,2024-05-24,5,2024,24
3,17971594208670851,17841425444516188,0.0,7.0,2024-05-24,5,2024,24
4,18062694892504235,17841425444516188,5.0,7.0,2024-05-24,5,2024,24


Fazendo a coluna de ID

In [112]:
FTbMidias__['id'] = FTbMidias__['id_midia'].astype(str) + \
    FTbMidias__['id_account'].astype(str) + FTbMidias__['year'].astype(str) \
          + FTbMidias__['period'].astype(str) + FTbMidias__['day'].astype(str)
FTbMidias__ = FTbMidias__[['id','id_midia', 'id_account', 'comments_count', 'like_count', 'extract_date', 'period', 'year', 'day']]
FTbMidias__.head()

,id,id_midia,id_account,comments_count,like_count,extract_date,period,year,day
0,18005273918369940178414254445161882024524,18005273918369940,17841425444516188,2.0,15.0,2024-05-24,5,2024,24
1,17846370906157882178414254445161882024524,17846370906157882,17841425444516188,1.0,13.0,2024-05-24,5,2024,24
2,18002752616172667178414254445161882024524,18002752616172667,17841425444516188,0.0,5.0,2024-05-24,5,2024,24
3,17971594208670851178414254445161882024524,17971594208670851,17841425444516188,0.0,7.0,2024-05-24,5,2024,24
4,18062694892504235178414254445161882024524,18062694892504235,17841425444516188,5.0,7.0,2024-05-24,5,2024,24


Tipagem dos dados

In [113]:
FTbMidias__.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              242 non-null    object 
 1   id_midia        242 non-null    int64  
 2   id_account      242 non-null    int64  
 3   comments_count  242 non-null    float64
 4   like_count      242 non-null    float64
 5   extract_date    242 non-null    object 
 6   period          242 non-null    int64  
 7   year            242 non-null    int64  
 8   day             242 non-null    int64  
dtypes: float64(2), int64(5), object(2)
memory usage: 17.1+ KB


In [114]:
FTbMidias__['extract_date'] = pd.to_datetime(FTbMidias__['extract_date'])
FTbMidias__['extract_date'] = FTbMidias__['extract_date'].dt.date

type_columns = {
    'id_midia': 'object',
    'id_account': 'object',
    'comments_count': 'int',
    'like_count': 'int',
    'period': 'object',
    'year': 'object',
    'day': 'object'
}

FTbMidias__ = FTbMidias__.astype(type_columns)
FTbMidias__.head()

,id,id_midia,id_account,comments_count,like_count,extract_date,period,year,day
0,18005273918369940178414254445161882024524,18005273918369940,17841425444516188,2,15,2024-05-24,5,2024,24
1,17846370906157882178414254445161882024524,17846370906157882,17841425444516188,1,13,2024-05-24,5,2024,24
2,18002752616172667178414254445161882024524,18002752616172667,17841425444516188,0,5,2024-05-24,5,2024,24
3,17971594208670851178414254445161882024524,17971594208670851,17841425444516188,0,7,2024-05-24,5,2024,24
4,18062694892504235178414254445161882024524,18062694892504235,17841425444516188,5,7,2024-05-24,5,2024,24


In [115]:
engine = connect_to_database('DW')
table_name = 'FTbMidias'
key_column = 'id'

if not engine: 
    print('conexão fail')
    sys.exit()

with engine.connect() as connection:
    for i, row in FTbMidias__.iterrows():
        query = text(f"select count(*) from {table_name} WHERE {key_column} = :value")
        result = connection.execute(query, parameters=dict(value=row[key_column]))
        count = result.scalar()

        if count > 0:
            print(f'Registro com {key_column} = {row[key_column]} já existe.')
        else: 
            row.to_frame().T.to_sql(table_name, con=connection, if_exists='append', index=False)
            print(f"Registro com {key_column} = {row[key_column]} inserido com sucesso.")
    connection.commit()
    connection.close()

Registro com id = 18005273918369940178414254445161882024524 já existe.
Registro com id = 17846370906157882178414254445161882024524 já existe.
Registro com id = 18002752616172667178414254445161882024524 já existe.
Registro com id = 17971594208670851178414254445161882024524 já existe.
Registro com id = 18062694892504235178414254445161882024524 já existe.
Registro com id = 17870243565059503178414254445161882024524 já existe.
Registro com id = 17897388878957061178414254445161882024524 já existe.
Registro com id = 18214911715272967178414254445161882024524 já existe.
Registro com id = 17905338809918494178414254445161882024524 já existe.
Registro com id = 17912491130893668178414254445161882024524 já existe.
Registro com id = 18020750125974446178414254445161882024524 já existe.
Registro com id = 17869447275058315178414254445161882024524 já existe.
Registro com id = 18030343753699664178414254445161882024524 já existe.
Registro com id = 18012618164496765178414254445161882024524 já existe.
Regist

Registro com id = 17945375882417545178414254445161882024524 já existe.
Registro com id = 17974769974769563178414254445161882024524 já existe.
Registro com id = 17961285041137957178414254445161882024524 já existe.
Registro com id = 17931701453556758178414254445161882024524 já existe.
Registro com id = 17972200513936539178414254445161882024524 já existe.
Registro com id = 18337501048046447178414254445161882024524 já existe.
Registro com id = 18014406560219838178414175304006162024524 já existe.
Registro com id = 18036730186652355178414175304006162024524 já existe.
Registro com id = 17926144523730968178414175304006162024524 já existe.
Registro com id = 17952648479723484178414175304006162024524 já existe.
Registro com id = 18012911482864852178414175304006162024524 já existe.
Registro com id = 18015261277785871178414175304006162024524 já existe.
Registro com id = 18213009469271569178414175304006162024524 já existe.
Registro com id = 17874981800973868178414175304006162024524 já existe.
Regist

## Tb DTbDescricaoInsights - Carregar dentro da DTbAccountInsights (O nome da DTbAccountInsights irá mudar para DTbDescricaoInsights)

BLOCO MOVIDO PARA O INICIO NO CARREGAMENTO DA DTbAccountInsights

## carregando a tabela FTbMidiasInsights

TbStoriesInsights

In [116]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbStoriesInsights.csv'
dataset_1 = pd.read_csv(file_path, sep='\t')
TbStoriesInsights = dataset_1
dataset_1.head(2)

,Unnamed: 0,id_midia,name,period,title,description,id,value,extract_date,period_extraction,year,day,id_tb_midias
0,0,18029303027098233,exits,lifetime,Saídas,Número de vezes que alguém saiu do story,18029303027098233/insights/exits/lifetime,12,2024-05-23,5,2024,23,1802930302709823320240523exitslifetime
1,1,18029303027098233,replies,lifetime,Respostas,Número total de respostas ao story,18029303027098233/insights/replies/lifetime,1,2024-05-23,5,2024,23,1802930302709823320240523replieslifetime


TbMidiasInsights

In [117]:
file_path = r'C:\Users\gabri\OneDrive\Documentos\Projetos\Instagram_data\api\silver\TbMidiasInsights.csv'
dataset_2 = pd.read_csv(file_path, sep='\t')
TbMidiasInsights = dataset_2
# TbMidiasInsights.rename(columns={'id': 'id_midia'}, inplace=True)
dataset_2.head(5)

,Unnamed: 0,id_midia,name,period,title,description,id,value,extract_date,period_extraction,year,day,id_tb_midias
0,0,18005273918369940,reach,lifetime,Contas alcançadas,O número de contas únicas que viram este reel ...,18005273918369940/insights/reach/lifetime,146.0,2024-05-24,5,2024,24,1800527391836994020240524reachlifetime
1,1,18005273918369940,saved,lifetime,Salvo,O número de vezes que seu reel foi salvo.,18005273918369940/insights/saved/lifetime,1.0,2024-05-24,5,2024,24,1800527391836994020240524savedlifetime
2,0,17846370906157882,reach,lifetime,Contas alcançadas,O número de contas únicas que viram este reel ...,17846370906157882/insights/reach/lifetime,99.0,2024-05-24,5,2024,24,1784637090615788220240524reachlifetime
3,1,17846370906157882,saved,lifetime,Salvo,O número de vezes que seu reel foi salvo.,17846370906157882/insights/saved/lifetime,0.0,2024-05-24,5,2024,24,1784637090615788220240524savedlifetime
4,0,18002752616172667,reach,lifetime,Contas alcançadas,O número de contas únicas que viram este reel ...,18002752616172667/insights/reach/lifetime,123.0,2024-05-24,5,2024,24,1800275261617266720240524reachlifetime


Unindo dimensão de descrição pelo NAME

Separando as colunas da dimensão

In [118]:
engine = connect_to_database('dw')
query = "select * from DTbDescricaoInsights"
DTbDescricaoInsights = pd.read_sql(query, engine)
DTbDescricaoInsights.head()

,id_insight,name,frequencia,title,description
0,1,audience_city,lifetime,Cidade do público,Cidades dos seguidores deste perfil
1,10,impressions,lifetime,Impressões,O número de vezes que seu story foi exibido na...
2,11,online_followers,lifetime,Seguidores online,Número total de seguidores deste perfil que es...
3,12,phone_call_clicks,day,Cliques em ligação telefônica,Número total de toques no link de ligação dest...
4,13,profile_views,day,Visualizações do perfil,Número total de usuários que visualizaram o pe...


In [119]:
TbStoriesInsights = pd.merge(TbStoriesInsights, DTbDescricaoInsights, on='name', how= 'inner')
TbStoriesInsights.rename(columns={'id_insight_x': 'id_insight'}, inplace=True)
TbMidiasInsights = pd.merge(TbMidiasInsights, DTbDescricaoInsights, on='name', how= 'inner')

Selecionando colunas

In [120]:
select_columns = [
    'id_midia',
    'id_insight',
    'extract_date',
    'period_extraction',
    'year',
    'day',
    'name',  # Renomear
    'value'
]

TbStoriesInsights = TbStoriesInsights[select_columns]
TbMidiasInsights = TbMidiasInsights[select_columns]

In [121]:
FTbMidiasInsights = pd.concat([TbStoriesInsights, TbMidiasInsights], ignore_index=True)
FTbMidiasInsights['id'] = FTbMidiasInsights['id_midia'].astype(str) + FTbMidiasInsights['id_insight'].astype(str) + \
                            FTbMidiasInsights['year'].astype(str) + FTbMidiasInsights['period_extraction'].astype(str) + \
                                FTbMidiasInsights['day'].astype(str)

FTbMidiasInsights = FTbMidiasInsights[['id','id_midia', 'id_insight', 'extract_date', 'period_extraction', 'year', 'day', 'name', 'value']]
FTbMidiasInsights.rename(columns={'name': 'definicao'}, inplace=True)
FTbMidiasInsights.head()

,id,id_midia,id_insight,extract_date,period_extraction,year,day,definicao,value
0,1802930302709823372024523,18029303027098233,7,2024-05-23,5,2024,23,exits,12.0
1,18029303027098233152024523,18029303027098233,15,2024-05-23,5,2024,23,replies,1.0
2,18029303027098233182024523,18029303027098233,18,2024-05-23,5,2024,23,taps_forward,36.0
3,18029303027098233172024523,18029303027098233,17,2024-05-23,5,2024,23,taps_back,1.0
4,18029303027098233102024523,18029303027098233,10,2024-05-23,5,2024,23,impressions,124.0


Tipagem de dados

In [122]:
FTbMidiasInsights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 470 entries, 0 to 469
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 470 non-null    object 
 1   id_midia           470 non-null    int64  
 2   id_insight         470 non-null    object 
 3   extract_date       470 non-null    object 
 4   period_extraction  470 non-null    int64  
 5   year               470 non-null    int64  
 6   day                470 non-null    int64  
 7   definicao          470 non-null    object 
 8   value              470 non-null    float64
dtypes: float64(1), int64(4), object(4)
memory usage: 33.2+ KB


unindo ambos os dataframes de stories e midias

In [123]:
FTbMidiasInsights['extract_date'] = pd.to_datetime(FTbMidiasInsights['extract_date'])
FTbMidiasInsights['extract_date'] = FTbMidiasInsights['extract_date'].dt.date

type_columns = {
    'id_midia': 'object',
    'id_insight': 'object',
    'period_extraction': 'object',
    'year': 'object',
    'day': 'object',
    'value': 'int'
}

FTbMidiasInsights = FTbMidiasInsights.astype(type_columns)
FTbMidiasInsights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 470 entries, 0 to 469
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 470 non-null    object
 1   id_midia           470 non-null    object
 2   id_insight         470 non-null    object
 3   extract_date       470 non-null    object
 4   period_extraction  470 non-null    object
 5   year               470 non-null    object
 6   day                470 non-null    object
 7   definicao          470 non-null    object
 8   value              470 non-null    int32 
dtypes: int32(1), object(8)
memory usage: 31.3+ KB


Carregando no banco

In [124]:
engine = connect_to_database('DW')
table_name = 'FTbMidiasInsights'
key_column = 'id'

if not engine: 
    print('conexão fail')
    sys.exit()

with engine.connect() as connection:
    for i, row in FTbMidiasInsights.iterrows():
        # Verificar se a chave estrangeira existe na tabela DTbMidias
        foreign_key = row['id_midia']
        query_fk = text("SELECT COUNT(*) FROM DTbMidias WHERE id_midia = :foreign_key")
        result_fk = connection.execute(query_fk, {'foreign_key': foreign_key})
        fk_count = result_fk.scalar()

        if fk_count == 0:
            print(f"A chave estrangeira {foreign_key} não existe na tabela DTbMidias. O registro não será inserido.")
            continue

        # Verificar se o registro já existe na tabela FTbMidiasInsights
        query_exists = text(f"SELECT COUNT(*) FROM {table_name} WHERE {key_column} = :value")
        result_exists = connection.execute(query_exists, {'value': row[key_column]})
        count = result_exists.scalar()

        if count > 0:
            print(f"Registro com {key_column} = {row[key_column]} já existe.")
        else:
            # Inserir o registro na tabela FTbMidiasInsights
            row.to_frame().T.to_sql(table_name, con=connection, if_exists='append', index=False)
            print(f"Registro com {key_column} = {row[key_column]} inserido com sucesso.")
    connection.commit()

Registro com id = 1802930302709823372024523 já existe.
Registro com id = 18029303027098233152024523 já existe.
Registro com id = 18029303027098233182024523 já existe.
Registro com id = 18029303027098233172024523 já existe.
Registro com id = 18029303027098233102024523 já existe.
Registro com id = 18029303027098233142024523 já existe.
Registro com id = 18005273918369940142024524 já existe.
Registro com id = 18005273918369940162024524 já existe.
Registro com id = 17846370906157882142024524 já existe.
Registro com id = 17846370906157882162024524 já existe.
Registro com id = 18002752616172667142024524 já existe.
Registro com id = 18002752616172667162024524 já existe.
Registro com id = 17971594208670851142024524 já existe.
Registro com id = 17971594208670851162024524 já existe.
Registro com id = 18062694892504235142024524 já existe.
Registro com id = 18062694892504235162024524 já existe.
Registro com id = 17870243565059503142024524 já existe.
Registro com id = 17870243565059503162024524 já e